In [1]:
\cd ../qenv

In [2]:
\l test.q

"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"36) .engine.logic.trade.Trade (UNIT) :READY"
`Global function for creating a new account
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                                                                                                                                                          
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
224    FAIL  1a) Prj;essTrade SELL: has agent hidden jxders, lvl1 size > q

In [4]:
setup:((!) . flip(
                (`instrument;(`iId`cntTyp`faceValue`mkprice`smul;enlist(0;`inverse;1;1000;1))); 
                (`account;(`aId`avail`bal`lng`srt`ft`rt`wit`time`froz;enlist(0;0;0;(0 1);(0 -1);0;0;0;z;0))); 
                (`inventory;(`aId`side`mm`upnl`ordQty`ordLoss`amt;flip(0 0;-1 1;0 0;0 0;0 0;0 0;10 10))); 
                (`feetier;(`ftId`vol`bal`ref;flip(0 1;0 0;0 0;0 0))); // Update Account
                (`orderbook;(
                        `iId`price`side`qty`hqty`iqty`vqty`time;
                        flip(0 0;1000 1001;2#1;2#1000;2#0;2#0;2#400;2#z)));
                (`order;(
                        `oId`aId`iId`ivId`side`lqty`oqty`price`reduce`dqty`okind`state`offset;
                        flip(til 4;4#0;4#0;4#enlist(0 1);4#1;4#100;4#100;4#1000;4#1b;4#1;4#1;4#0;0 110 220 330))
                );
                (`risktier;(`rtId`amt`lev;flip(0 1;50000 250000;125 100))) // Update Account
            ));

In [5]:
.engine.testutils.SwitchSetupModels[setup];

In [6]:
x:.model.Order . (`oId`aId`iId`ivId`side`oqty`lqty`price`reduce`dqty;flip(4 5;0 0;0 0;((0 1);(0 1));1 1;300 100;300 100;1000 1000;11b;300 100));

In [7]:
l:0!?[`.engine.model.orderbook.Orderbook;(
        (in;`side;x`side);
        (>;(+;`qty;(+;`hqty;(+;`iqty;`vqty)));0);
        (|;(<;(+\;`qty);sum[x`oqty]);
        (=;`i;(*:;`i))));0b;()];

In [8]:
aqty:sum[l[`qty`iqty`hqty`vqty]];
thresh:sums[aqty];
rp:min[(sum x`oqty;first[aqty])]^((thresh-prev[thresh])-(thresh-sum x`oqty));

In [9]:
o:?[`.engine.model.order.Order;(
            (=;`okind;1);
            (in;`price;l[`price] where (rp>0));
            (in;`state;(0 1));(>;`oqty;0));0b;()];

In [10]:
parse"?[((offset<=rp)and((lqty+offset)<=rp));1;?[(sums[offset]<=rp);2;state]]"

?
(&;((';~:;>);`offset;`rp);((';~:;>);(+;`lqty;`offset);`rp))
1
(?;((';~:;>);(+\;`offset);`rp);2;`state)


In [11]:
s:0!((`price`side xkey l) ij (`price`side xgroup o));
s[`rp]:rp;
msk:raze[.util.PadM[{x#1}'[count'[s`oId]]]];

In [12]:
so:ungroup s

In [13]:
x`oqty

300 100


In [46]:
so`rp

400 400 400 400


In [56]:
count `iId`oId`time`aId`side`okind`price`dqty`shft`state`reduce`lqty`offset

13


In [60]:
so`iId`oId`time`aId`side`okind`price`dqty`shft`state`reduce`lqty`offset

`.engine.model.instrument.Instrument$0 0 0 0
1 2 0 3
0N 0N 0N 0Nz
`.engine.model.account.Account$0 0 0 0
1 1 1 1
1 1 1 1
1000 1000 1000 1000
1 1 1 1
`long$()
0 0 0 0
1111b
100 100 100 100
110 220 0 330


In [71]:
raze '[rp]

400


In [14]:
newstate:{[state;offset;rp;lqty]rp:first rp;?[((offset<=rp)and((lqty+offset)<=rp));1;?[(sums[offset]<=rp);2;state]]};

In [32]:
//(?;(&;((';~:;>);`offset;`rp);((';~:;>);(+;`lqty;`offset);`rp));1;(?;((';~:;>);(+\;`offset);`rp);2;`state))

// Update Orders
// Derive the new orders from the ungrouped 
// state. //TODO check new state
no:![?[so;();0b;`iId`oId`time`aId`ivId`side`okind`price`dqty`state`reduce`lqty`offset`shft!(
        `iId;`oId;`time;`aId;`ivId;`side;`okind;`price;`dqty;`state;`reduce;
        (max;(enlist;(?;(>;`rp;`lqty);(-;(+;`lqty;`offset);`rp);`lqty);0)); // lqty
        (.util.Clip;(-;`offset;`rp)); // offset
        (.util.Clip;(+;(-;`offset;`rp);`lqty)) // shft // todo test
    )];();0b;`dqty`shft`state!(
        (.util.Clip;(?;(&;(<;`dqty;`lqty);(>;`lqty;0));`dqty;`lqty));
        (+;`lqty;`offset);
        (newstate;`state;`offset;`rp;`lqty) // TODO update state
    )]

In [33]:
no

iId oId time aId ivId side okind price dqty state reduce lqty offset shft
-------------------------------------------------------------------------
0   1        0   1    1    1     1000  0    1     1      0    0      0   
0   2        0   1    1    1     1000  0    1     1      0    0      0   
0   0        0   1    1    1     1000  0    1     1      0    0      0   
0   3        0   1    1    1     1000  1    1     1      30   0      30  


In [38]:
nl:flip select
   hqty:max[(hqty-rp;0)],
   iqty:lqty-dqty,
   vqty:qty,
   qty:min[((qty-rp)-(sum[so[`lqty] - no[`lqty]]);0)],
   iId:iId,
   time:time,
   price:price
   from s

In [40]:
select qty from s

qty 
----
1000


In [98]:
s ij (`price`side xgroup no)

price side iId     qty  hqty iqty vqty time oId     aId     ivId    slprice sprice trig tif okind   oskind reduce state   oqty            dqty    lqty     slqty offset  einst rp  shft    
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1000  1    0 0 0 0 1000 0    0    400       1 2 0 3 0 0 0 0 1 1 1 1                         1 1 1 1        1111b  0 0 0 0 100 100 100 100 0 0 0 1 0 0 0 30       0 0 0 0       400 0 0 0 30


In [48]:
?[s ij (`price`side xgroup no);();0b;`price`side`hqty`iqty`vqty`qty`time!(
    `price;`side;
    (.util.Clip;(-;`hqty;`rp));
    ((';sum);(-;`lqty;`dqty));
    ((';sum);`lqty);
    (.util.Clip;(-;(-;`qty;`rp);(sum;(-;so[`lqty];no[`lqty])))); // TODO
    (max;x`time)
    )]

price side hqty iqty vqty qty time
----------------------------------
1000  1    0    29   30   230 -0w 


In [24]:
so`lqty

100 100 100 100


In [37]:
?[no;();0b;`time`price`qty`reduce`ismaker`side`iId`oId`ivId`aId!(
    `time;`price;so[`lqty] - no[`lqty];
    `reduce;(=;`okind;1);`side;`iId;`oId;`ivId;`aId)]

time price qty reduce ismaker side iId oId ivId aId
---------------------------------------------------
     1000  100 1      1       1    0   1   1    0  
     1000  100 1      1       1    0   2   1    0  
     1000  100 1      1       1    0   0   1    0  
     1000  70  1      1       1    0   3   1    0  


In [35]:
ft

time price qty reduce ismaker side iId oId ivId aId
---------------------------------------------------
     1000  100 1      1       1    0   1   1    0  
     1000  100 1      1       1    0   2   1    0  
     1000  100 1      1       1    0   0   1    0  
     1000  70  1      1       1    0   3   1    0  


In [15]:
s:0!((`price`side xkey l) ij (`price`side xgroup o));
s[`rp]:rp;
msk:raze[.util.PadM[{x#1}'[count'[s`oId]]]];

// Pad s into a matrix
// for faster operations
pdcl:`oId`side`price`offset`oqty`lqty`reduce`aId`state;
(s pdcl):.util.PadM'[s pdcl];

// Useful counts 
maxN:max count'[s`offset];
tmaxN:til maxN;
numLvls:count[s`offset];

// TODO new display qty
// Calculate new shifts and max shifts
shft:sum[s`offset`lqty]; // the sum of the order offsets and lqty
mxshft:{$[x>1;max[y];x=1;y;0]}'[maxN;shft]; // the max shft for each price

// TOOD update comments
// The delta in the visual qty is equal to sum of the change in the open display qty
// and the total fill qty that isnt used to fill the hqty or iqty of the previous
// queue instantiation.
// The qty on the other hand is equal to the sum of the amount of the visual qty
// that isn't made up of the new dqty. 
// Derive the new quantitites that are representative of the change in the s
// of the orders and orderbook when the given trade occurs.
nhqty:          .util.Clip[(-/)s`hqty`rp]; // Derive the new hidden qty
nlqty:          .util.Clip[{?[x>z;(y+z)-x;y]}'[s`rp;s`lqty;s`offset]]; // TODO faster/fix
ndqty:          .util.Clip[{?[((x<y) and (y>0));x;y]}'[s[`dqty];nlqty]]; // TODO faster/fix
niqty:          sum'[nlqty-ndqty]; // The new order invisible qty = lqty - display qty
displaydlt:     (ndqty-s[`dqty]); // Derive the change in the order display qty
lqtydlt:        (nlqty-s[`lqty]); // Derive the change in the order lqty
hqtydlt:        (nhqty-s[`hqty]); // Derive the change in hidden order qty
noffset:        .util.Clip[((-/)s`offset`rp)]; // Subtract the replaced amount and clip<0 (offset includes hqty)
nqty:           .util.Clip[((-/)s`qty`rp)-(sum'[lqtydlt]+hqtydlt)]; // qty -rp - qty attributed to other qtys
nvqty:          nqty+sum'[ndqty]; // Derive the new visible qty as order display qty + the new qty from above
nshft:          nlqty+noffset; // 
nmxshft:        {$[x>1;max[y];x=1;y;0]}'[maxN;nshft]; // the max shft for each price TODO make faster
nfilled:        s[`oqty] - nlqty; // New amount that is filled (in total)
accdlts:        s[`lqty] - nlqty; // The new amounts that will attribute to fills.
nobupd:         count[nhqty];

// Derive the non agent qty's from the s such that quantities
// such as the visible resultant trades can be derived etc.
notAgentQty: .util.PadM[raze'[flip(
        0^s[`hqty]; // hidden qty
        0^(asc'[s[`offset]][;0] - 0^s[`hqty]); // first offset
        .util.Clip[0^s[`offset][;1_(tmaxN)] - 0^shft[;-1_(tmaxN)]]; // middle offset + shft
        .util.Clip[s[`vqty]-mxshft] // last qty - maximum shift
    )]];

// Derive the splt which is the combination of the lqty of all orders at 
// a level with the interspaced display qty etc. which is used to derive the
// disparate quantities of trades that occur at a given price level.
splt:{$[count[x];1_(raze raze'[(2#0),(0^x);y]);y]}'[s`lqty;notAgentQty];

// Derives the non-zero trade qtys that occur as a result of the replaced amount 
// returns the quantities by price level.
tqty:{s:sums[y];q:.util.Clip[?[(x-s)>=0;y;x-(s-y)]];q where[q>0]}'[s`rp;splt]; 

In [36]:
tqty

100 10 100 100 90


In [38]:
notAgentQty

0 0 0 0 230 0


In [35]:
notAgentQty

0 0 10 0 230 0


In [13]:
s:0!((`price`side`iId xkey l) lj (`price`side`iId xgroup o));
msk:raze[.util.PadM[{x#1}'[count'[s`oId]]]];

s[`aqty]:aqty;
s[`thresh]:thresh;
s[`rp]:rp;
// Pad s into a matrix
// for faster operations
//pdcl:`oId`side`price`offset`oqty`lqty`reduce`aId`state;
//(s pdcl):.util.PadM'[s pdcl];

// Useful counts 
s[`maxN]:max count'[s`offset];
s[`tmaxN]:til'[s`maxN];
s[`numLvls]:count[s`offset];

In [14]:
flip s

price  | 1000   
side   | 1      
iId    | 0      
qty    | 1000   
hqty   | 0      
iqty   | 0      
vqty   | 200    
time   |        
oId    | 1 2    
aId    | 0 0    
ivId   | 1 1    
slprice|        
sprice |        
trig   |        
tif    |        
okind  | 1 1    
oskind |        
reduce | 11b    
state  | 0 0    
oqty   | 200 200
dqty   | 1 1    
lqty   | 200 200
slqty  |        
offset | 0 210  
einst  |        
aqty   | 1200   
thresh | 1200   
rp     | 400    
maxN   | 2      
tmaxN  | 0 1    
numLvls| 1      


In [51]:
/ hqty:.util.Clip[(-/)(hqty,rp)]
/ nlqty:.util.Clip[{?[x>z;(y+z)-x;y]}'[rp;lqty;offset]]

In [15]:
so:ungroup s

In [ ]:
//agent fills is set to orders?
.engine.model.order.Order:([oId:`long$()] 
        aId:`.engine.model.account.Account$();iId:`.engine.model.instrument.Instrument$();
        ivId:`.engine.model.inventory.Inventory$();price:`long$();slprice:`long$();side:`long$();
        sprice:`long$();trig:`long$();tif:`long$();okind:`long$();oskind:`long$();reduce:`boolean$();
        state:`long$();oqty:`long$();dqty:`long$();lqty:`long$();slqty:`long$();offset:`long$();
        einst:`long$();time:`datetime$());

In [16]:
.event.Order:{x:.event.prep[x];([]kind:`order;time:x`time;datum:flip x`oId`time`aId`side`okind`price`slqty`lprice`slprice`state`reduce`trig`einst;aId:7h$x`aId)}

In [2]:
\c 600 600

In [24]:
no:flip
    update
    dqty:max[(?[((dqty<lqty) and (lqty>0));dqty;lqty])],
    shft:lqty+offset,
    state:?[((offset<=rp)and((lqty+offset)<=rp));1;?[(sums[offset]<=rp);2;state]]
    from update 
    iId:iId,
    oId:oId,
    time:time,
    aId:aId,
    side:side,
    okind:okind,
    price:price,
    offset:max[(offset-rp;0)], // todo non functional
    lqty:max[(?[rp>lqty;(lqty+offset)-rp;lqty];0)], 
    reduce:reduce,
    slprice:slprice,
    slqty:slqty,
    state:state,
    reduce:reduce,
    trig:trig,
    einst:einst
    from so

iId oId time aId side okind price dqty state reduce lqty offset shft
--------------------------------------------------------------------
0   1        0   1    1     1000  0    1     1      0    0      0   
0   2        0   1    1     1000  0    1     1      0    0      0   
0   0        0   1    1     1000  0    1     1      0    0      0   
0   3        0   1    1     1000  1    1     1      30   0      30  


In [ ]:
nqty:           .util.Clip[((-/)s`qty`rp)-(sum'[lqtydlt]+hqtydlt)]; // qty -rp - qty attributed to other qtys
nvqty:          nqty+sum'[ndqty]; // Derive the new visible qty as order display qty + the new qty from above

In [20]:
nl:flip select
   hqty:max[(hqty-rp;0)],
   iqty:lqty-dqty,
   vqty:qty,
   qty:min[((qty-rp)-(sum[lqtydlt]+hqtydlt);0)],
   iId:iId,
   time:time,
   price:price
   from s

In [21]:
nl

hqty | 0      
iqty | 199 199
vqty | 0      
qty  | 0      
iId  | 0      
time |        
price| 1000   


In [ ]:
f:?[`]

In [22]:
f:select //book fills
    time:time,
    price:price,
    qty:lqty - lqty, // todo dependency
    reduce:reduce,
    ismaker:okind=1,
    side:side,
    iId:iId,
    oId:oId,
    ivId:ivId,
    aId:aId
    from so

In [29]:
f

time price qty reduce ismaker side iId oId ivId aId
---------------------------------------------------
     1000  0   1      1       1    0   1   1    0  
     1000  0   1      1       1    0   2   1    0  


In [16]:
// Derive the non agent qty's from the s such that quantities
// such as the visible resultant trades can be derived etc.
notAgentQty: .util.PadM[raze'[flip(
        0^s[`hqty]; // hidden qty
        0^(s[`offset][;0] - 0^s[`hqty]); // first offset
        .util.Clip[0^s[`offset][;1_(tmaxN)] - 0^shft[;-1_(tmaxN)]]; // middle offset + shft
        .util.Clip[s[`vqty]-mxshft] // last qty - maximum shift
    )]];

// Derive the splt which is the combination of the lqty of all orders at 
// a level with the interspaced display qty etc. which is used to derive the
// disparate quantities of trades that occur at a given price level.
splt:{$[count[x];1_(raze raze'[(2#0),(0^x);y]);y]}'[s`lqty;notAgentQty];

// Derives the non-zero trade qtys that occur as a result of the replaced amount 
// returns the quantities by price level.
tqty:{s:sums[y];q:.util.Clip[?[(x-s)>=0;y;x-(s-y)]];q where[q>0]}'[s`rp;splt]; 

In [17]:
tqty

110 100 10 100 80


In [36]:
nagent:{[hqty;offset;shft;vqty;qty;mxshft] 
    show offset;
    (hqty;offset[0]-hqty;.util.Clip[(1_offset)-(1_shft)];.util.Clip[(vqty+hqty+qty)-mxshft])
    }

In [41]:
select n:nagent'[hqty;offset;shft;vqty;qty;mxshft] from s

110 220 0 330


n              
---------------
0 110 0 0 0 970


In [43]:
select offset, qty, vqty, rp from so

offset qty  vqty rp 
--------------------
110    1000 400  400
220    1000 400  400
0      1000 400  400
330    1000 400  400


In [64]:
\l test.q

"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"40) .engine.logic.fill.Fill (UNIT) :READY"
`Function for deriving the exec cost from the qty and the price
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                            
-------------------------------------------------------------
228    ERROR INVERSE:flat to long: UPL: 0, RPL:0 ONE POSITION
229    ERROR INVERSE:long to flat: UPL: 0, RPL:0 ONE POSITION
"                                                                                                                                                                                       

In [65]:
.bam.ivx

fId    | 0                       1                      
time   | 2020.12.24T09:19:33.417 2020.12.24T09:19:33.417
price  | 1000                    1000                   
qty    | 100                     100                    
reduce | 0                       1                      
ismaker| 0                       1                      
side   | 1                       -1                     
iId    | 0                       0                      
oId    | 0                       0                      
ivId   | 0                       1                      
aId    | 0                       0                      


In [71]:
select .engine.logic.contract.inverse.ExecCost[price;qty;iId.smul] from .bam.ivx

smul| 10000000 10000000


In [78]:
parse"select 
    ivId.amt, 
    ivId.totEnt, 
    .engine.logic.contract.inverse.AvgPriceDeriv[side;price;qty;ivId.totEnt;iId.smul],
    .engine.logic.contract.inverse.ExecCost[price;qty;iId.smul] from .bam.ivx"

?
`.bam.ivx
()
0b
`amt`totEnt`smul`smul1!(`ivId.amt;`ivId.totEnt;(`.engine.logic.contract.inverse.AvgPriceDeriv;`side;`price;`qty;`ivId.totEnt;`iId.smul);(`.engine.logic.contract.inverse.ExecCost;`price;`qty;`iId.smul))


In [85]:
?[.bam.ivx;();0b;`avgPrice`execCost`ordLoss`ordVal`ordQty`amt`totEnt!(
    (first;(.engine.logic.contract;`iId.cntTyp;enlist`AvgPriceDeriv;`side;`price;`qty;`ivId.totEnt;`iId.smul)); 
    (.engine.logic.contract;`iId.cntTyp;enlist`ExecCost;`price;`qty;`iId.smul); 
    (-;`ivId.ordLoss;(.engine.logic.contract;`iId.cntTyp;enlist`Loss;`iId.mkprice;`qty;`price));
    (-;`ivId.ordVal;((*/);`qty;`price)); // TODO contract dependent
    (-;`ivId.ordQty;`qty);
    (+;`ivId.amt;`qty);
    ((+/);`ivId.totEnt;`qty))]

avgPrice| 100               100              
execCost| 10000000 10000000 10000000 10000000
ordLoss | 0 0               0 0              
ordVal  | -100000000        -100000000       
ordQty  | -100              -100             
amt     | 110               110              
totEnt  | 210               210              


In [61]:
.engine.logic.contract.inverse.UnrealizedPnl          :{[amt;isignum;avgPrice;markPrice;faceValue;multiplier]
    show multiplier;
    show isignum;
    show faceValue;
    show avgPrice;
    7h$((((faceValue%avgPrice)-(faceValue%markPrice))*(amt*isignum))*multiplier)
    };

In [ ]:
amt;isignum;avgPrice;markPrice;faceValue;multiplier

In [63]:
select avgPrice, .engine.logic.contract.inverse.UnrealizedPnl[amt;side;avgPrice;iId.mkprice;iId.faceValue;iId.smul] from .bam.iv1

1 1
1 -1
1 1
100 100
100 100


avgPrice| 100 100 100 100
smul    | 1  1    -1 -1  


In [ ]:
nhqty:          .util.Clip[(-/)s`hqty`rp]; // Derive the new hidden qty
nlqty:          .util.Clip[{?[x>z;(y+z)-x;y]}'[s`rp;s`lqty;s`offset]]; // TODO faster/fix
ndqty:          .util.Clip[{?[((x<y) and (y>0));x;y]}'[s[`dqty];nlqty]]; // TODO faster/fix
niqty:          sum'[nlqty-ndqty]; // The new order invisible qty = lqty - display qty
displaydlt:     (ndqty-s[`dqty]); // Derive the change in the order display qty
lqtydlt:        (nlqty-s[`lqty]); // Derive the change in the order lqty
hqtydlt:        (nhqty-s[`hqty]); // Derive the change in hidden order qty
noffset:        .util.Clip[((-/)s`offset`rp)]; // Subtract the replaced amount and clip<0 (offset includes hqty)
nqty:           .util.Clip[((-/)s`qty`rp)-(sum'[lqtydlt]+hqtydlt)]; // qty -rp - qty attributed to other qtys
nvqty:          nqty+sum'[ndqty]; // Derive the new visible qty as order display qty + the new qty from above
nshft:          nlqty+noffset; // 
nmxshft:        {$[x>1;max[y];x=1;y;0]}'[maxN;nshft]; // the max shft for each price TODO make faster
nfilled:        s[`oqty] - nlqty; // New amount that is filled (in total)
accdlts:        s[`lqty] - nlqty; // The new amounts that will attribute to fills.
nobupd:         count[nhqty];